Importing dependencies and loading csv file

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
ds=pd.read_csv("/content/drive/MyDrive/ML Projects/Twitter_live_Sentiment_Analysis/sentiment140.csv", encoding = "ISO-8859-1")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id"     1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [6]:
ds.drop(columns=['id"','date','flag','user'],inplace=True)


In [7]:
y=ds['target']

In [8]:
ds.head()


,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [9]:
ds.duplicated().sum() #checking for duplicates

16309

In [10]:
ds=ds.drop_duplicates(keep='first')

Target value for all tweets is 0 or 4 at which implies negative and positive

In [11]:
ds = ds.sample(frac = 1)#shuffling

In [12]:
ds.head()

,target,text
387601,0,"good morning, unfortunally !"
1056276,4,Omg the school looks soo cool! haha their deco...
294047,0,"Nah, Really got bored... Just sitting and watc..."
1009367,4,"#ralphonelottery coffee and cake, or pint and ..."
1019563,4,Follow Friday: @awkwardfamily heaps funny


In [39]:
ds_split=np.array_split(ds,40)#splitting 1.6 Million tweets into 40 equal parts

In [40]:
ds_split[0]['target'].value_counts()

4    19837
0    19756
Name: target, dtype: int64

In [41]:
df=ds_split[0] #df stores the fraction of our original dataframe
df.head()

,target,text
387601,0,"good morning, unfortunally !"
1056276,4,Omg the school looks soo cool! haha their deco...
294047,0,"Nah, Really got bored... Just sitting and watc..."
1009367,4,"#ralphonelottery coffee and cake, or pint and ..."
1019563,4,Follow Friday: @awkwardfamily heaps funny


In [42]:
df=df.reset_index()

In [43]:
df.drop(columns=['index'],inplace=True)

In [44]:
df.head()

,target,text
0,0,"good morning, unfortunally !"
1,4,Omg the school looks soo cool! haha their deco...
2,0,"Nah, Really got bored... Just sitting and watc..."
3,4,"#ralphonelottery coffee and cake, or pint and ..."
4,4,Follow Friday: @awkwardfamily heaps funny


In [45]:
df['target'].value_counts()

4    19837
0    19756
Name: target, dtype: int64

Preprocessing

In [46]:
!pip install nltk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
import nltk

In [48]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

In [50]:
from nltk.stem.porter import PorterStemmer

In [51]:
import re
import string

In [52]:

def pp(text):

  text=re.sub(r'https?:\/\/.*[\r\n]*','', text)
  text=re.sub(r'#','',text)

  tokenizer = TweetTokenizer(preserve_case=False,strip_handles=True, reduce_len=True)
  text= tokenizer.tokenize(text)

  y=[]
  for i in text:
    if i.isalnum():
      y.append(i)

  text=y[:]
  y.clear()

  stop_words = set(stopwords.words('english'))
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  for i in text:
    if i not in stop_words and i not in punc:
      y.append(i)

  text=y[:]
  y.clear()

  ps=PorterStemmer()
  for i in text:
    y.append(ps.stem(i))

  return " ".join(y)


In [53]:
pp(df['text'][1])

'omg school look soo cool haha decor prommm'

In [54]:
df['text'][1]

'Omg the school looks soo cool! haha their decorating it for the after prommm. '

In [55]:

df['pp_text']=df[df.columns[1]].apply(pp)
df.loc[df['target'] == 4, 'target'] = 1

In [56]:
df.head()

,target,text,pp_text
0,0,"good morning, unfortunally !",good morn unfortun
1,1,Omg the school looks soo cool! haha their deco...,omg school look soo cool haha decor prommm
2,0,"Nah, Really got bored... Just sitting and watc...",nah realli got bore sit watch star cricket cha...
3,1,"#ralphonelottery coffee and cake, or pint and ...",ralphonelotteri coffe cake pint crisp
4,1,Follow Friday: @awkwardfamily heaps funny,follow friday heap funni


Dropping empty rows

In [57]:
df.dropna(inplace=True)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39593 entries, 0 to 39592
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   target   39593 non-null  int64 
 1   text     39593 non-null  object
 2   pp_text  39593 non-null  object
dtypes: int64(1), object(2)
memory usage: 928.1+ KB


Model

In [59]:
y=df['target'].values

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf= TfidfVectorizer(max_features=3750)
X=tfidf.fit_transform(df['pp_text']).toarray()
len(tfidf.vocabulary_)

3750

In [90]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size =0.3,
                                                    random_state=1)


In [62]:
# import pandas as pd
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# models = {
#     'Multinomial Naive Bayes': MultinomialNB(),
#     'Logistic Regression': LogisticRegression(),
#     'Random Forest': RandomForestClassifier(),
#     'SVM': SVC(),
#     'K-Nearest Neighbors': KNeighborsClassifier(),
#     'Decision Tree': DecisionTreeClassifier()
# }

# results_df = pd.DataFrame(columns=['Accuracy', 'F1 Score', 'Precision', 'Recall'])

# for model_name, model in models.items():

#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)

#     #evaluation metrics
#     accuracy = accuracy_score(y_test, y_pred)
#     f1 = f1_score(y_test, y_pred)
#     precision = precision_score(y_test, y_pred)
#     recall = recall_score(y_test, y_pred)

#     results_df.loc[model_name] = [accuracy, f1, precision, recall]


In [63]:
# print(results_df)

In [64]:
# from sklearn.svm import SVC
# svcm = SVC()
# svc=svcm.fit(X_train, y_train)
# y_pred=svc.predict(X_test)


In [94]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def modelMetrics(y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)

  print(accuracy)
  print(f1)
  print(precision)
  print(recall)

In [95]:
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()
mnb=mnb.fit(X_train,y_train)
y_pred=mnb.predict(X_test)
modelMetrics(y_pred)

0.7309311331874053
0.7345955821292145
0.7250819672131148
0.7443621676203298


In [96]:
from sklearn.linear_model import LogisticRegression

lr= LogisticRegression()
lr=lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
modelMetrics(y_pred)


0.7422124936858057
0.7504889178617992
0.7274881516587678
0.7749915853248065


SVM Classification model gives the best accuracy for our use case

SVM pickle file is too large to train and upload on git, therefore selecting the next best model - LR

In [97]:
import pickle
pickle.dump(tfidf,open('sentiment_analysis_vectorizer.pkl','wb'))
pickle.dump(lr,open('sentiment_analysis_model.pkl','wb'))
